<a href="https://colab.research.google.com/github/CCTV-Smartview-Analytics/Project-Notebook/blob/main/CCTV_Smartview_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CCTV Live Streams

### These are some public cctv live streams of some countries

<br>

Canada(Toranto) - http://72.138.175.115/mjpg/video.mjpg?camera=1

Canada(Sault Ste. Marie) - http://142.112.106.49:84/mjpg/video.mjpg

India(Bengaluru) - http://117.215.128.17:82/nphMotionJpeg?Resolution=640x480&Quality=Motion

Germany(Spiekeroog) - http://5.61.162.124/mjpg/video.mjpg?camera=1

France(Vorey) - http://88.166.245.77:81/mjpg/video.mjpg

France(Saint-Denis) - http://77.135.210.108/mjpg/video.mjpg

USA(Dallas) - http://98.172.44.107:8090/mjpg/video.mjpg

USA(Dallas) - http://166.251.105.190/mjpg/video.mjpg

USA(Savannah) - http://73.182.17.160:8081/mjpg/1/video.mjpg

Japan(Tokyo) - http://118.22.23.185/nphMotionJpeg?Resolution=640x480&Quality=Standard

Japan(Tokyo) - http://219.101.88.67:60001/nphMotionJpeg?Resolution=640x480&Quality=Clarity

## Install Necessary Modules

In [1]:
!pip install ultralytics opencv-python colorthief webcolors
!pip install colab-xterm
%load_ext colabxterm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.6/115.6 kB 3.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Modules

In [3]:
from ultralytics import YOLO
import cv2, time, os, csv, requests
from colorthief import ColorThief
import webcolors
from webcolors import rgb_to_name
import datetime
from dateutil.tz import gettz
from PIL import Image as im
from moviepy.editor import *
from google.colab.patches import cv2_imshow

In [13]:
with open('close_color_finder.py','w') as f:
  f.write(requests.get('https://raw.githubusercontent.com/CCTV-Smartview-Analytics/Project-Notebook/main/close_color_finder.py').text)
from close_color_finder import convert_rgb_to_names

In [ ]:
# %cd /content

# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="T5uMjNxNNTTeAUIUq0fl")
# project = rf.workspace("master-dataset-curation").project("cctv-curation-dataset-poc")
# version = project.version(13)
# dataset = version.download("yolov8")


In [ ]:
# %cd /content/CCTV-Curation-Dataset-PoC-13
# model = YOLO('yolov8n.pt')
# results = model.train(data='coco128.yaml', epochs=100, imgsz=640)

## Yolo Detection

In [21]:
%cd /content
start = time.time()

# Replace the below url with the above url to choose desired stream
# [requests.get('http://118.22.23.185/nphControlCamera?Direction=Darker&Resolution=640x480&Quality=Standard&Mode=JPEG&RPeriod=65535&Size=STD&PresetOperation=Move&Language=0') for cnt in range(0,5)]

cap = cv2.VideoCapture('https://player.vimeo.com/progressive_redirect/playback/328732843/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=bb455a6abdfae56a5cb1f7a8a7e0e309128d832a3b3e15796c6922b7a771c3ac')#('https://player.vimeo.com/progressive_redirect/playback/210741646/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=37077c1af632c7cb144dffed7e04dcc10ede6939be1b57ead875b6b8874b9cfb')#('https://player.vimeo.com/progressive_redirect/playback/328732843/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=bb455a6abdfae56a5cb1f7a8a7e0e309128d832a3b3e15796c6922b7a771c3ac')#('/content/abcd.mp4')
cap = cv2.VideoCapture('/content/abcd.mp4')

model = YOLO('yolov8n.pt')#('/content/CCTV-Curation-Dataset-PoC-13/runs/detect/train/weights/best.pt')

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc('X','V','I','D')
size = (width,height)
fps = 10#cap.get(5)


time_before_start = datetime.datetime.now(tz=gettz('Asia/Kolkata'))
time_now = time_before_start
frame_no = 0
add_sec = 0

# folder_path = os.path.join('')'/content/drive/MyDrive/'
folder_path = os.path.join(os.path.abspath('.'),'drive','MyDrive')

output = cv2.VideoWriter(folder_path + datetime.datetime.now().strftime('%d-%m-%Y') + '.avi', fourcc, fps, size, -1)
class_names = model.names
box_csv = open('output_summary.csv','w')
writer = csv.writer(box_csv)
writer.writerow(['Frame No.', 'X1', 'Y1', 'X2', 'Y2', 'Confidence', 'Class', 'Color'])

try:
  while cap.isOpened():

    # Read each frame from video stream
    ret, frame = cap.read()

    frame_no += 1

    # In case of no stream stop execution
    if not ret:
      break

    # Predicted result of each frame
    results = model.predict(frame, conf=0.5)

    # Result with various bounding boxes for each frame
    for i in results:

      # Check if there is any detection
      if i.boxes.data.tolist():
        # Save the results with predicted class and normalised bounding box
        results[0].save_txt('abc.txt')
        # print(i.masks.data.tolist())
        # for each of the object get its bounding box values
        for detection in i.boxes.data.tolist():
          x1, y1, x2, y2, conf, cls = detection

          cropped_frame = frame[int(y1):int(y2),int(x1):int(x2)]
          cropped_frame = frame[int((y1+y2)/2):int(((y1+y2)/2)+20),int((x1+x2)/2):int(((x1+x2)/2)+20)]
          cropped_img = im.fromarray(cropped_frame)
          cropped_img.save('__temp__.png')
          cv2_imshow(cropped_frame)
          dominant_color = ColorThief('__temp__.png').get_color(quality=1)
          print(dominant_color)

          writer.writerow([frame_no, x1, y1, x2, y2, conf, cls, convert_rgb_to_names(dominant_color)])
          text_size = cv2.getTextSize(str(time_now.time().strftime('%X')), cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
          text_x = max((x1 + x2)//2 - text_size[0] // 2, 0)
          text_y = max((y1 + y2)//2 - text_size[1] // 2, 0)

          # cv2.putText(frame , text, coordinates, fontstyle, scaling_factor, color, thickness, line_style)
          cv2.putText(frame, str(datetime.datetime.now().strftime('%X')), (int((x1+x2)/2), int((y1+y2)/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)

          # Create directory for each objects
          if not os.path.exists(class_names[int(cls)]):
            os.mkdir(class_names[int(cls)])

          break
        frame = cv2.resize(frame, size)

        # Save the numpy array as image with their corresponding timestamps
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        data = im.fromarray(rgb_frame)
        cur_time = time.time()
        data.save(f'{class_names[int(cls)]}/{cur_time}.png',)
        # print(ColorThief(f'{class_names[int(cls)]}/{cur_time}.png').get_color(quality=1))
        # Write the video file with the frame
        output.write(frame)

      else:
        print('No Detection')

    # cv2_imshow(frame)

    # if cv2.waitKey(1) == ord('q'):
    #   break

    if frame_no % int(fps) == 0:
      add_sec += 1
      time_now = time_before_start + datetime.timedelta(0,add_sec)

except InterruptedError:
  pass

finally:
  output.release()
  cap.release()
  # cv2.destroyAllWindows()

print(time.time() - start)

/content

0: 640x384 1 person, 141.7ms
Speed: 4.4ms preprocess, 141.7ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 384)


(68, 96, 148)

0: 640x384 1 person, 149.9ms
Speed: 4.5ms preprocess, 149.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)


(67, 95, 147)

0: 640x384 1 person, 143.7ms
Speed: 3.5ms preprocess, 143.7ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)


(66, 95, 148)

0: 640x384 1 person, 147.9ms
Speed: 5.0ms preprocess, 147.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)


(64, 97, 148)

0: 640x384 1 person, 161.6ms
Speed: 4.3ms preprocess, 161.6ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)


...

(157, 147, 122)

0: 640x384 1 person, 136.2ms
Speed: 4.6ms preprocess, 136.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)


(155, 146, 121)

0: 640x384 1 person, 146.9ms
Speed: 4.7ms preprocess, 146.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)


(154, 144, 119)

0: 640x384 1 person, 155.0ms
Speed: 5.7ms preprocess, 155.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 384)


(155, 146, 122)

0: 640x384 1 person, 170.6ms
Speed: 4.2ms preprocess, 170.6ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 384)


(154, 146, 123)

0: 640x384 (no detections), 151.3ms
Speed: 5.5ms preprocess, 151.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)
No Detection

0: 640x384 (no detections), 159.7ms
Speed: 3.4ms preprocess, 159.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
No Detection

0: 640x384 (no detections), 149.5ms
Speed: 4.1ms preprocess, 149.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)
No Detection

0: 640x384 (no detections), 157.8ms
Speed: 4.8ms preprocess, 157.8ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)
No Detection

0: 640x384 (no detections), 166.7ms
Speed: 4.3ms preprocess, 166.7ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
No Detection

0: 640x384 (no detections), 152.9ms
Speed: 3.4ms preprocess, 152.9ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
No Detection

0: 640x384 (no detections), 153.9ms
Speed: 3.8ms preprocess, 153.9ms inference, 0.8ms postproce

In [45]:
from scipy.spatial import KDTree
from webcolors import (
    CSS3_HEX_TO_NAMES,
    hex_to_rgb,
)

def convert_rgb_to_names(rgb_tuple):
    # a dictionary of all the hex and their respective names in CSS3
    names = []
    rgb_values = []
    for color_hex, color_name in CSS3_HEX_TO_NAMES.items():
        names.append(color_name)
        rgb_values.append(hex_to_rgb(color_hex))

    kdt_db = KDTree(rgb_values)
    distance, index = kdt_db.query(rgb_tuple)
    return f'{names[index]}'


## Read Stored summary video along with CSV file

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/box_csv.csv')

cap = cv2.VideoCapture('/content/drive/MyDrive/output1.avi')

fr_cnt = 0

while cap.isOpened():
  ret, frame = cap.read()

  if not ret:
    break

  fr_cnt += 1

  for i in df[df['Frame No.'] == fr_cnt].iterrows():
    cv2.rectangle(frame, (int(i[1].X1), int(i[1].Y1)), (int(i[1].X2), int(i[1].Y2)), (255,0,0))
    cv2_imshow(frame)

cap.release()

## Encode Video in proper format to be displayed in colab

In [ ]:
path="/content/drive/MyDrive/output1.avi"

clip=VideoFileClip(path)
clip.ipython_display(width=size[0],maxduration=120)

## Generate Video from Images

In [ ]:
import os

def create_video():
  li = os.listdir('car')
  li = sorted(li)
  out = cv2.VideoWriter('/content/drive/MyDrive/summary/output_image.avi', fourcc, 30, size)
  for i in li:
    frame = cv2.imread('car/'+i)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    out.write(rgb_frame)
  out.release()

create_video()

## Colab Terminal

In [ ]:
%xterm

# start = time.time()

# # Replace the below url with the above url
# cap = cv2.VideoCapture('http://98.172.44.107:8090/mjpg/video.mjpg')#('https://player.vimeo.com/progressive_redirect/playback/210741646/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=37077c1af632c7cb144dffed7e04dcc10ede6939be1b57ead875b6b8874b9cfb')#('https://player.vimeo.com/progressive_redirect/playback/328732843/rendition/360p/file.mp4?loc=external&oauth2_token_id=1747418641&signature=bb455a6abdfae56a5cb1f7a8a7e0e309128d832a3b3e15796c6922b7a771c3ac')#('/content/abcd.mp4')

# model = YOLO('yolov8n.pt')

# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc('m','p','4','v')
# size = (width,height)
# fps = cap.get(5)


# time_before_start = datetime.datetime.now(tz=gettz('Asia/Kolkata'))
# time_now = time_before_start
# frame_no = 0
# add_sec = 0

# output = cv2.VideoWriter('output.mp4', fourcc, fps, size)
# class_names = model.names

# try:
#   while cap.isOpened():

#     # Read each frame from video stream
#     ret, frame = cap.read()

#     frame_no += 1

#     # In case of no stream stop execution
#     if not ret:
#       break

#     # Predicted result of each frame
#     results = model.predict(frame, conf=0.5)

#     # Result with various bounding boxes for each frame

#     for i in results:

#       # Check if there is any detection
#       if i.boxes.data.tolist():
#         # Save the results with predicted class and normalised bounding box
#         results[0].save_txt('abc.txt')

#         # for each of the object get its bounding box values
#         for detection in i.boxes.data.tolist():
#           x1, y1, x2, y2, conf, cls = detection

#           # Calculate the center coordinates of the bounding box
#           center_x = int((x1 + x2) / 2)
#           center_y = int((y1 + y2) / 2)

#           # Draw the timestamp near the object
#           timestamp_text = str(time_now.time().strftime('%X'))
#           text_size = cv2.getTextSize(timestamp_text, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1)[0]
#           text_x = max(center_x - text_size[0] // 2, 0)
#           text_y = max(center_y - text_size[1] // 2, 0)
#           cv2.putText(frame, timestamp_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)

#           # Draw the bounding box
#           cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

#           # Create directory for each object class
#           class_dir = class_names[int(cls)]
#           if not os.path.exists(class_dir):
#               os.mkdir(class_dir)

#           # Save the frame with the bounding box and timestamp
#           frame_with_box = frame.copy()
#           cv2.putText(frame_with_box, timestamp_text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1, cv2.LINE_AA)
#           cv2.rectangle(frame_with_box, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
#           cv2.imwrite(f'{class_dir}/{time.time()}.png', frame_with_box)

#       # Write the video file with the frame
#       output.write(frame)
# except InterruptedError:
#   pass

# finally:
#   output.release()
#   cap.release()
#   cv2.destroyAllWindows()

# print(time.time() - start)



In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('output.mp4','rb').read()
data_url = "data:video/avi;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/avi">
</video>
""" % data_url)

In [ ]:
import datetime
from dateutil.tz import gettz


a= datetime.datetime.now(tz=gettz('Asia/Kolkata'))
b = a + datetime.timedelta(0,5)

print(a.time().strftime('%X'))
print(b.time().strftime('%X'))

## List available streams

In [ ]:
import requests

res = requests.get('https://cdn.sourceb.in/bins/7PR1Y6ImJB/0')

for i in res.text.split('\n'):
  print(i.strip())


In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git


In [ ]:
!cd /content/yolov9/

In [ ]:
!pip install -r yolov9/requirements.txt

In [ ]:
%cd yolov9

In [ ]:
!python val.py --data data/coco.yaml --img 640 --batch 32 --conf 0.001 --iou 0.7 --device cpu --weights './yolov9-c-converted.pt' --save-json --name yolov9_c_c_640_val

In [ ]:
!python detect.py --source './data/images/horses.jpg' --img 640 --device 0 --weights './yolov9-c-converted.pt' --name yolov9_c_c_640_detect

In [ ]:
%cd /content
!unzip CCTV-Curation-Dataset-PoC-13-20240317T184212Z-001 -d /content

In [ ]:
datetime.datetime.now()

In [ ]:
!ffmpeg -i output1.avi output.mp4